In [3]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from efficientnet_pytorch import EfficientNet
import torchvision.transforms as transforms
import torchvision.datasets as datasets

In [ ]:
# Load configuration
with open('../config.yaml', 'r') as f:
    config = yaml.safe_load(f)

# Configuration parameters
batch_size = config['train']['batch_size']
num_epochs = config['train']['num_epochs']
learning_rate = config['train']['learning_rate']

In [2]:
!pip install efficientnet_pytorch

  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet_pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16428 sha256=aac8d44644c6eba2f1f2d6af30e1cb286e0e5e93509498a63ec062ebfc6ddf11
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
Successfully built efficientnet_pytorch


In [5]:
# Load the processed datasets
train_dataset = torch.load('/kaggle/input/cifar-10-raw-processed/data/processed/train_dataset.pth') # ..\\data\\processed\\train_dataset.pth
test_dataset = torch.load('/kaggle/input/cifar-10-raw-processed/data/processed/test_dataset.pth') # ..\\data\\processed\\test_dataset.pth

In [6]:
# Define data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [7]:
# Initialize the model, loss function, and optimizer
model = EfficientNet.from_pretrained('efficientnet-b0')
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b0-355c32eb.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b0-355c32eb.pth
100%|██████████| 20.4M/20.4M [00:00<00:00, 159MB/s]

Loaded pretrained weights for efficientnet-b0


In [16]:
import torch.nn as nn
from torch.utils.data import DataLoader
from tqdm import tqdm

# Assuming you have already defined your model, optimizer, and criterion
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.2, momentum=0.1, nesterov=True)
loss_func = torch.nn.MSELoss()

# Assuming you have a DataLoader named train_loader
for epoch in range(num_epochs): # Specify the num_epochs as your need
    model.train()
    running_loss = 0.0
    with tqdm(train_loader, desc=f"Epoch {epoch+1}") as t:
        for inputs, labels in t:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            t.set_postfix(loss=loss.item())  # Update progress bar with loss

    avg_loss = running_loss / len(train_loader)
    train_loss_history.append(avg_loss)
    print(f"Epoch {epoch+1}, Loss: {avg_loss:.4f}")


Epoch 1: 100%|██████████| 1563/1563 [06:22<00:00,  4.09it/s, loss=0.723]


Epoch 1, Loss: 0.6991


Epoch 2: 100%|██████████| 1563/1563 [06:21<00:00,  4.09it/s, loss=0.787]

Epoch 2, Loss: 0.6135


In [24]:
# Save the trained model
torch.save(model.state_dict(), '../results/models/efficientnet_b0.pth')